In [1]:
# from BCBio.GFF import GFFExaminer

https://gffpandas.readthedocs.io/en/latest/tutorial.html

In [2]:
import gffpandas.gffpandas as gffpd
import pandas as pd

In [82]:
annotation = gffpd.read_gff3('YPS1009_S288C_v4_polished.gff')
annot_s288c = gffpd.read_gff3('saccharomyces_cerevisiae_R64-3.gff')

C:\Users\user\anaconda3\lib\site-packages\gffpandas\gffpandas.py:32: DtypeWarning: Columns (1,2,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_table(self._gff_file, comment='#',


In [6]:
annotation.df['type'].value_counts()

CDS                                7178
gene                               6707
mRNA                               6707
noncoding_exon                      493
tRNA_gene                           321
tRNA                                321
transposable_element_gene            99
transposable_element                 99
snoRNA                               77
snoRNA_gene                          77
plus_1_translational_frameshift      47
five_prime_UTR_intron                24
ncRNA_gene                           23
ncRNA                                23
snRNA_gene                            6
snRNA                                 6
uORF                                  6
telomeric_repeat                      3
intein_encoding_region                1
Name: type, dtype: int64

In [85]:
annot_s288c.df['type'].value_counts()

mRNA                                  11119
CDS                                    7066
gene                                   6607
ARS                                     543
noncoding_exon                          489
long_terminal_repeat                    384
intron                                  378
tRNA                                    299
tRNA_gene                               299
ARS_consensus_sequence                  196
transposable_element_gene                91
transposable_element                     91
region                                   78
snoRNA_gene                              77
snoRNA                                   77
telomere                                 64
LTR_retrotransposon                      50
plus_1_translational_frameshift          47
X_element                                32
centromere                               32
telomeric_repeat                         31
X_element_combinatorial_repeat           28
rRNA                            

In [7]:
sgd_s288c = pd.read_csv('chr_features_s288c.csv', index_col=0)

In [8]:
sgd_s288c.sum()

Verified ORF                                                              5208
long terminal repeat                                                       384
ARS                                                                        352
tRNA gene                                                                  275
transposable element gene                                                   91
snoRNA gene                                                                 77
LTR retrotransposon                                                         50
rRNA gene                                                                   22
ncRNA gene                                                                  22
Total Features                                                            7944
Chromosome Length (bp)       230,218813,184316,6201,531,933576,874270,1611,...
dtype: object

In [9]:
CDS_df = annotation.filter_feature_of_type(['CDS'])

In [10]:
TE_df_2 = annotation.filter_feature_of_type(['transposable_element'])

In [11]:
TE_df = annotation.filter_feature_of_type(['transposable_element_gene'])
sno_df = annotation.filter_feature_of_type(['snoRNA_gene'])
tRNA_df = annotation.filter_feature_of_type(['tRNA_gene'])
gene_df = annotation.filter_feature_of_type(['gene'])

In [12]:
CDS_attr = CDS_df.attributes_to_columns()

In [13]:
CDS_attr.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'ID', 'Name', 'Parent', 'extra_copy_number',
       'orf_classification', 'protein_id'],
      dtype='object')

In [14]:
mRNA_df = annotation.filter_feature_of_type(['mRNA'])
mRNA_attr = mRNA_df.attributes_to_columns()

In [15]:
mRNA_attr['valid_ORF'].value_counts()

True     6177
False     530
Name: valid_ORF, dtype: int64

In [16]:
gene_attr = gene_df.attributes_to_columns()
gene = gene_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'low_identity', 'orf_classification',
       'partial_mapping', 'sequence_ID', 'valid_ORFs']]

In [17]:
gene_attr['valid_ORFs'].value_counts()
# so valid_ORFs in gene does count itself and I should look carefully at the 439 genes with no valid ORF

1    6175
0     530
Name: valid_ORFs, dtype: int64

In [18]:
TE_attr = TE_df.attributes_to_columns()
sno_attr = sno_df.attributes_to_columns()
tRNA_attr = tRNA_df.attributes_to_columns()

In [19]:
tRNA_attr.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'Ontology_term',
       'copy_num_ID', 'coverage', 'curie', 'dbxref', 'display',
       'extra_copy_number', 'gene', 'sequence_ID'],
      dtype='object')

In [20]:
TE = TE_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'sequence_ID', 'valid_ORFs']]

sno = sno_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'sequence_ID']]

tRNA = tRNA_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'sequence_ID']]

In [21]:
# This code just convert the chromosome numbering to integer and remove mitochondrial genes.
def lat_to_arabic(DF):
    Latin_to_arabic = {'I': 1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':9, 'X':10,
                      'XI':11, 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 'XVI':16, 'Mito': 'mitoch', 'MANY': 7}
    seq_id = DF['seq_id']
    ch = []
    contig = []
    for i in seq_id:
        chrom, cont = i.split('_')
        contig.append(cont)
        chrom = chrom.replace('chr', '')
        if '-' in chrom:
            chrom, e = chrom.split('-')
        ch.append(Latin_to_arabic[chrom])
    DF['Chromosome'] = ch
    DF['Contig'] = contig
    return DF

In [22]:
gene = lat_to_arabic(gene)

C:\Users\user\AppData\Local\Temp\ipykernel_32584\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_32584\3764198105.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Contig'] = contig


In [23]:
TE = lat_to_arabic(TE)
sno = lat_to_arabic(sno)
tRNA = lat_to_arabic(tRNA)

C:\Users\user\AppData\Local\Temp\ipykernel_32584\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_32584\3764198105.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Contig'] = contig
C:\Users\user\AppData\Local\Temp\ipykernel_32584\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [24]:
gene.head()

,seq_id,start,end,Name,Alias,Note,Ontology_term,score,strand,phase,...,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs,Chromosome,Contig
0,chrII-2_tig00004927,8297,10119,YBR298C,"MAL31,MAL3T,MALT,maltose%20permease",Maltose%20permease%3B%20high-affinity%20maltos...,"GO:0000017,GO:0005352,GO:0071944,SO:0000704",.,-,.,...,Maltose%20permease,1,MAL31,None,Verified,None,0.946,0,2,tig00004927
3,chrII-2_tig00004927,10675,10894,YBR298C-A,None,Putative%20protein%20of%20unknown%20function%3...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Putative%20protein%20of%20unknown%20function,1,None,None,Uncharacterized,None,0.981,0,2,tig00004927
6,chrII-2_tig00004927,10985,12717,YBR299W,"MAL32,MAL3S,MALS,alpha-glucosidase%20MAL32",Maltase%20%28alpha-D-glucosidase%29%3B%20induc...,"GO:0000025,GO:0004575,GO:0005886,GO:0005987,GO...",.,+,.,...,Maltase%20%28alpha-D-glucosidase%29,1,MAL32,None,Verified,None,0.981,0,2,tig00004927
9,chrII-2_tig00004927,14242,14700,YGR293C,None,Dubious%20open%20reading%20frame%3B%20unlikely...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Dubious%20open%20reading%20frame,2,None,None,Dubious,None,0.989,0,2,tig00004927
12,chrII-2_tig00004927,14660,15022,YGR294W,"PAU12,seripauperin%20PAU12",Protein%20of%20unknown%20function%3B%20member%...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Protein%20of%20unknown%20function,1,PAU12,None,Verified,None,0.983,1,2,tig00004927


In [25]:
def clean20(DF):
    note = DF['Note']
    dis = DF['display']
    clean = []
    display = []
    for n in note:
        clean.append(n.replace('%20', ' '))
    for d in dis:
        display.append(str(d).replace('%20', ' '))
    DF['Note'] = clean
    DF['display'] = display
    return DF

In [26]:
gene = clean20(gene)
TE = clean20(TE)
sno = clean20(sno)
tRNA = clean20(tRNA)

C:\Users\user\AppData\Local\Temp\ipykernel_32584\741321415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Note'] = clean
C:\Users\user\AppData\Local\Temp\ipykernel_32584\741321415.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['display'] = display


# TE

In [27]:
TE['seq_id'].value_counts()

chrMANY_tig00000038     54
chrXII-2_tig00004934    14
chrIV_tig00000001       12
chrV_tig00000015         8
chrII_tig00000010        2
chrVI_tig00004938        2
chrXV_tig00000007        2
chrIII_tig00004936       1
chrI_tig00000033         1
chrVIII_tig00000017      1
chrXVI-2_tig00000040     1
chrX_tig00001098         1
Name: seq_id, dtype: int64

In [28]:
TE['Chromosome'].value_counts()

7     54
12    14
4     12
5      8
2      2
6      2
15     2
3      1
1      1
8      1
16     1
10     1
Name: Chromosome, dtype: int64

In [29]:
TE['valid_ORFs'].value_counts()

1    61
0    38
Name: valid_ORFs, dtype: int64

In [30]:
sum(sgd_s288c['transposable element gene'])

91

In [31]:
TE_count = TE['Chromosome'].value_counts().to_frame()
TE_count = TE_count.join(sgd_s288c['transposable element gene'])
TE_count = TE_count.rename(columns = {'Chromosome': 'yps1009 TE', 'transposable element gene': 's288c TE'})

In [32]:
TE_count

,yps1009 TE,s288c TE
7,54,10
12,14,8
4,12,17
5,8,4
2,2,6
6,2,2
15,2,8
3,1,2
1,1,2
8,1,4


# snoRNA

In [33]:
sno_count = sno['Chromosome'].value_counts().to_frame()
sno_count = sno_count.join(sgd_s288c['snoRNA gene'])
sno_count = sno_count.rename(columns = {'Chromosome': 'yps1009 snoRNA', 'snoRNA gene': 's288c snoRNA'})

In [34]:
sno_count

,yps1009 snoRNA,s288c snoRNA
13,13,13
15,11,11
12,7,7
7,6,6
16,6,6
5,5,5
14,5,5
11,5,5
10,5,5
3,4,4


# tRNA

In [71]:
tRNA_count = tRNA['Chromosome'].value_counts().to_frame()
tRNA_count = tRNA_count.join(sgd_s288c['tRNA gene'])
tRNA_count = tRNA_count.rename(columns = {'Chromosome': 'yps1009 tRNA', 'tRNA gene': 's288c tRNA'})

In [78]:
tRNA

,seq_id,start,end,Name,Alias,Note,Ontology_term,score,strand,phase,copy_num_ID,coverage,curie,display,extra_copy_number,gene,sequence_ID,Chromosome,Contig
201,chrIII_tig00004936,82917,82988,YNCC0001C,tE%28UUC%29C,Glutamine tRNA %28tRNA-Glu%29%2C predicted by ...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,-,.,YNCC0001C_0,1.0,SGD:S000006549,Glutamine tRNA %28tRNA-Glu%29%2C predicted by ...,0,None,1.0,3,tig00004936
207,chrIII_tig00004936,84660,84773,YNCC0002W,"SUP53,tL%28CAA%29C",Leucine tRNA %28tRNA-Leu%29%2C predicted by tR...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,+,.,YNCC0002W_1,1.0,SGD:S000006637,Leucine tRNA %28tRNA-Leu%29%2C predicted by tR...,1,SUP53,1.0,3,tig00004936
287,chrIII_tig00004936,117379,117450,YNCC0004C,"SUF2,tP%28AGG%29C",Proline tRNA %28tRNA-Pro%29%2C predicted by tR...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,-,.,YNCC0004C_0,1.0,SGD:S000006678,Proline tRNA %28tRNA-Pro%29%2C predicted by tR...,0,SUF2,1.0,3,tig00004936
293,chrIII_tig00004936,121186,121259,YNCC0005W,tN%28GUU%29C,Asparagine tRNA %28tRNA-Asn%29%2C predicted by...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,+,.,YNCC0005W_0,1.0,SGD:S000006668,Asparagine tRNA %28tRNA-Asn%29%2C predicted by...,0,None,1.0,3,tig00004936
296,chrIII_tig00004936,121186,121259,YNCN0003W,tN%28GUU%29N1,Asparagine tRNA %28tRNA-Asn%29%2C predicted by...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,+,.,YNCN0003W_0,1.0,SGD:S000006673,Asparagine tRNA %28tRNA-Asn%29%2C predicted by...,0,None,1.0,3,tig00004936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21819,chrX_tig00001098,516144,516214,YNCJ0024W,"SUF23,tG%28GCC%29J2",Glycine tRNA %28tRNA-Gly%29%3B can mutate to s...,"GO:0005829,GO:0006414,GO:0030371,GO:0030533,GO...",.,+,.,YNCJ0024W_0,1.0,SGD:S000006584,Glycine tRNA %28tRNA-Gly%29,0,SUF23,1.0,10,tig00001098
21831,chrX_tig00001098,522533,522604,YNCJ0025W,"HSX1,tR%28CCU%29J",Arginine tRNA %28tRNA-Arg%29%2C predicted by t...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,+,.,YNCJ0025W_0,1.0,SGD:S000006707,Arginine tRNA %28tRNA-Arg%29%2C predicted by t...,0,HSX1,1.0,10,tig00001098
21837,chrX_tig00001098,525148,525219,YNCJ0026W,tD%28GUC%29J4,Aspartate tRNA %28tRNA-Asp%29%2C predicted by ...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,+,.,YNCJ0026W_0,1.0,SGD:S000006539,Aspartate tRNA %28tRNA-Asp%29%2C predicted by ...,0,None,1.0,10,tig00001098
21843,chrX_tig00001098,526596,526684,YNCJ0027C,"SUP4,tY%28GUA%29J2",Tyrosine tRNA %28tRNA-Tyr%29%2C predicted by t...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,-,.,YNCJ0027C_0,1.0,SGD:S000006782,Tyrosine tRNA %28tRNA-Tyr%29%2C predicted by t...,0,SUP4,1.0,10,tig00001098


In [73]:
tRNA10_yps = tRNA[tRNA['Chromosome'] == 10] 

,seq_id,start,end,Name,Alias,Note,Ontology_term,score,strand,phase,copy_num_ID,coverage,curie,display,extra_copy_number,gene,sequence_ID,Chromosome,Contig
20961,chrX_tig00001098,54683,54755,YNCJ0001C,tT%28AGU%29J,Threonine tRNA %28tRNA-Thr%29%2C predicted by ...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,-,.,YNCJ0001C_0,1.0,SGD:S000006743,Threonine tRNA %28tRNA-Thr%29%2C predicted by ...,0,None,1.0,10,tig00001098
21084,chrX_tig00001098,111547,111618,YNCG0022W,tE%28UUC%29G3,Glutamate tRNA %28tRNA-Glu%29%2C predicted by ...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,-,.,YNCG0022W_0,1.0,SGD:S000006555,Glutamate tRNA %28tRNA-Glu%29%2C predicted by ...,0,None,1.0,10,tig00001098
21087,chrX_tig00001098,111547,111618,YNCJ0002C,tE%28UUC%29J,Glutamate tRNA %28tRNA-Glu%29%2C predicted by ...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,-,.,YNCJ0002C_0,1.0,SGD:S000006557,Glutamate tRNA %28tRNA-Glu%29%2C predicted by ...,0,None,1.0,10,tig00001098
21254,chrX_tig00001098,192901,192973,YNCJ0005C,tA%28AGC%29J,Alanine tRNA %28tRNA-Ala%29%2C predicted by tR...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,-,.,YNCJ0005C_0,1.0,SGD:S000006514,Alanine tRNA %28tRNA-Ala%29%2C predicted by tR...,0,None,1.0,10,tig00001098
21257,chrX_tig00001098,194025,194096,YNCJ0006W,tD%28GUC%29J1,Aspartate tRNA %28tRNA-Asp%29%2C predicted by ...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,+,.,YNCJ0006W_0,1.0,SGD:S000006536,Aspartate tRNA %28tRNA-Asp%29%2C predicted by ...,0,None,1.0,10,tig00001098
21296,chrX_tig00001098,223226,223298,YNCJ0008W,tR%28ACG%29J,Arginine tRNA %28tRNA-Arg%29%2C predicted by t...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,+,.,YNCJ0008W_0,1.0,SGD:S000006702,Arginine tRNA %28tRNA-Arg%29%2C predicted by t...,0,None,1.0,10,tig00001098
21485,chrX_tig00001098,343652,343740,YNCJ0010W,"SUP7,tY%28GUA%29J1",Tyrosine tRNA %28tRNA-Tyr%29%2C predicted by t...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,+,.,YNCJ0010W_0,1.0,SGD:S000006781,Tyrosine tRNA %28tRNA-Tyr%29%2C predicted by t...,0,SUP7,1.0,10,tig00001098
21489,chrX_tig00001098,344819,344890,YNCJ0011W,tR%28UCU%29J1,Arginine tRNA %28tRNA-Arg%29%2C predicted by t...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,+,.,YNCJ0011W_0,1.0,SGD:S000006714,Arginine tRNA %28tRNA-Arg%29%2C predicted by t...,0,None,1.0,10,tig00001098
21492,chrX_tig00001098,344901,344972,YNCJ0012W,tD%28GUC%29J2,Aspartate tRNA %28tRNA-Asp%29%2C predicted by ...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,+,.,YNCJ0012W_0,1.0,SGD:S000006537,Aspartate tRNA %28tRNA-Asp%29%2C predicted by ...,0,None,1.0,10,tig00001098
21514,chrX_tig00001098,363855,363926,YNCJ0013C,tD%28GUC%29J3,Aspartate tRNA %28tRNA-Asp%29%2C predicted by ...,"GO:0005829,GO:0006414,GO:0030533,SO:0000704",.,-,.,YNCJ0013C_0,1.0,SGD:S000006538,Aspartate tRNA %28tRNA-Asp%29%2C predicted by ...,0,None,1.0,10,tig00001098


# GENE

In [38]:
gene.columns

Index(['seq_id', 'start', 'end', 'Name', 'Alias', 'Note', 'Ontology_term',
       'score', 'strand', 'phase', 'copy_num_ID', 'coverage', 'curie',
       'display', 'extra_copy_number', 'gene', 'low_identity',
       'orf_classification', 'partial_mapping', 'sequence_ID', 'valid_ORFs',
       'Chromosome', 'Contig'],
      dtype='object')

In [39]:
gene = gene[['Name', 'Chromosome', 'Contig', 'start', 'end', 'Alias', 'Note', 'Ontology_term',
       'coverage', 'curie', 'display', 'extra_copy_number',
       'gene', 'low_identity', 'orf_classification', 'partial_mapping',
       'sequence_ID', 'valid_ORFs']]

In [40]:
gene['orf_classification'].value_counts()

Verified           5249
Uncharacterized     737
Dubious             719
Name: orf_classification, dtype: int64

In [41]:
dubious = gene[gene['orf_classification'] == 'Dubious']
dubious['valid_ORFs'].value_counts()

1    524
0    195
Name: valid_ORFs, dtype: int64

In [42]:
uncharacterized = gene[gene['orf_classification'] == 'Uncharacterized']
uncharacterized['valid_ORFs'].value_counts()

1    564
0    173
Name: valid_ORFs, dtype: int64

# Verified ORF

In [43]:
verified = gene[gene['orf_classification'] == 'Verified']

In [44]:
verified['valid_ORFs'].value_counts()

1    5087
0     162
Name: valid_ORFs, dtype: int64

In [45]:
verified.columns

Index(['Name', 'Chromosome', 'Contig', 'start', 'end', 'Alias', 'Note',
       'Ontology_term', 'coverage', 'curie', 'display', 'extra_copy_number',
       'gene', 'low_identity', 'orf_classification', 'partial_mapping',
       'sequence_ID', 'valid_ORFs'],
      dtype='object')

In [46]:
verified_valid = verified[verified['valid_ORFs'] == '1']
verified_unvalid = verified[verified['valid_ORFs'] == '0']

verified_valid_count = verified_valid['Chromosome'].value_counts().to_frame()
verified_valid_count = verified_valid_count.rename(columns={'Chromosome': 'Number valid genes'})
verified_unvalid_count = verified_unvalid['Chromosome'].value_counts().to_frame()
verified_unvalid_count = verified_unvalid_count.rename(columns={'Chromosome': 'Number unvalid genes'})

In [47]:
verified_unvalid.head(2)

,Name,Chromosome,Contig,start,end,Alias,Note,Ontology_term,coverage,curie,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs
0,YBR298C,2,tig00004927,8297,10119,"MAL31,MAL3T,MALT,maltose%20permease",Maltose permease%3B high-affinity maltose tran...,"GO:0000017,GO:0005352,GO:0071944,SO:0000704",0.986,SGD:S000000502,Maltose permease,1,MAL31,None,Verified,None,0.946,0
6,YBR299W,2,tig00004927,10985,12717,"MAL32,MAL3S,MALS,alpha-glucosidase%20MAL32",Maltase %28alpha-D-glucosidase%29%3B inducible...,"GO:0000025,GO:0004575,GO:0005886,GO:0005987,GO...",0.986,SGD:S000000503,Maltase %28alpha-D-glucosidase%29,1,MAL32,None,Verified,None,0.981,0


In [48]:
unval_dic = {}
for ind, row in verified_unvalid.iterrows():
    if row['Chromosome'] in unval_dic:
        unval_dic[row['Chromosome']] = unval_dic[row['Chromosome']] + [row['Name']]
    else:
        unval_dic[row['Chromosome']] = [row['Name']]    

In [49]:
verified_sum = verified_valid_count.join(verified_unvalid_count)

In [52]:
tmp = []
for ind in verified_sum.index:
    tmp.append(unval_dic[ind])

In [53]:
verified_sum['unvalid genes'] = tmp

In [54]:
verifiedS288C = pd.read_excel('verified_orf_s288c.xlsx')
verifiedS288C.set_index("Gene.secondaryIdentifier", inplace = True)

In [55]:
# This code just convert the chromosome numbering to integer and remove mitochondrial genes.
Latin_to_arabic = {'I': 1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':9, 'X':10,
                  'XI':11, 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 'XVI':16, 'mt': 'mitoch'}
chLat = verifiedS288C['Gene.chromosome.primaryIdentifier']
ch = []
for chrom in chLat:
    chrom = chrom.replace('chr', '')
    ch.append(Latin_to_arabic[chrom])
verifiedS288C['S288C Chromosome'] = ch

In [56]:
tmp = []
count = []
for Ch in verified_sum.index:
    s288 = verifiedS288C[verifiedS288C['S288C Chromosome'] == Ch]
    yps = verified[verified['Chromosome'] == Ch]
    tmp.append(list(set(s288.index) - set(yps['Name'])))
    count.append(len(list(set(s288.index) - set(yps['Name']))))

In [57]:
verified_sum['Number of missing genes'] = count
verified_sum['Missing verified genes'] = tmp

In [58]:
verified_sum

,Number valid genes,Number unvalid genes,unvalid genes,Number of missing genes,Missing verified genes
4,673,9,"[YDL247W, YNL331C, YDL227C, YDL080C, YDL039C, ...",5,"[YDR385W, YDR542W, YDL248W, YDR545W, YDL244W]"
15,467,14,"[YOR388C, YOR249C, YOR244W, YOR245C, YCR106W, ...",6,"[YOR133W, YOR247W, YOR390W, YOR394W, YOR255W, ..."
7,457,5,"[YGL243W, YGL131C, YAR027W, YGR140W, YBR298C]",11,"[YGR294W, YGR200C, YGL133W, YGL136C, YGL051W, ..."
12,432,13,"[YGL263W, YLL063C, YLL057C, YLL052C, YLR119W, ...",9,"[YLR132C, YLR160C, YLR466W, YLR461W, YLR157C, ..."
16,405,8,"[YOR394W, YPL282C, YMR323W, YPL274W, YPL255W, ...",5,"[YPL283C, YPR204W, YPL281C, YPL279C, YPL280W]"
13,394,8,"[YDL248W, YJR161C, YML118W, YMR019W, YMR104C, ...",5,"[YMR323W, YMR322C, YMR105C, YMR325W, YML132W]"
2,360,7,"[YBR298C, YBR299W, YGR295C, YBR031W, YBR073W, ...",3,"[YBL108C-A, YBR302C, YBL111C]"
14,326,18,"[YNL322C, YNL318C, YNL317W, YNL316C, YNL315C, ...",15,"[YNL329C, YNL332W, YNL333W, YNL328C, YNR076W, ..."
10,316,12,"[YJL222W, YJL219W, YJL222W, YJL217W, YJL077C, ...",5,"[YJR159W, YJR161C, YJR160C, YJR158W, YJL223C]"
11,291,12,"[YKR102W, YKR103W, YKR104W, YKR105C, YKL161C, ...",2,"[YKL224C, YKR106W]"


In [79]:
verified_sum.sum

Number valid genes                                                      5087
Number unvalid genes                                                     162
unvalid genes              [YDL247W, YNL331C, YDL227C, YDL080C, YDL039C, ...
Number of missing genes                                                   83
Missing verified genes     [YDR385W, YDR542W, YDL248W, YDR545W, YDL244W, ...
dtype: object

In [59]:
verified_sum.to_excel('verified_genes_summary.xlsx')

# Blast

In [60]:

# Import Biopython tools for running local BLASTX
from Bio.Blast.Applications import NcbiblastnCommandline

# Chromosome IV
[YDR385W, YDR542W, YDL248W, YDR545W, YDL244W]

In [61]:
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YDR038C_ENA5_genomic.fsa', out='Blast_results/YDR038C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')

In [62]:
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YDR039C_ENA2_genomic.fsa', out='Blast_results/YDR039C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)

STDOUT: 
STDERR: 


In [63]:
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YDR545W_YRF1-1_genomic.fsa', out='Blast_results/YDR545W.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)

STDOUT: 
STDERR: 


In [64]:
verified[verified['Name'] == 'YDR545W']

,Name,Chromosome,Contig,start,end,Alias,Note,Ontology_term,coverage,curie,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs
9902,YDR545W,5,tig00000015,680,6415,"YRF1-1,Y'%20element%20ATP-dependent%20helicase...",Helicase encoded by the Y' element of subtelom...,"GO:0000722,GO:0003678,GO:0005634,SO:0000704",0.999,SGD:S000002953,Helicase encoded by the Y' element of subtelom...,0,YRF1-1,None,Verified,None,0.924,1


Chromosome 4: 2 near centromere, 1 at the very end of teleomere
at centromere: ENA2 and ENA5 : next to each other, Na(+)-exporting P-type ATPase ENA2
ENA5 align with 90% identity to contig tig00000001 IV
ENA2 align to exactly the same region of contig IV, with 90% identity, duplication in s288C?

YRF1 align to contig >tig00000015 on chromosome V with 97% identity, at the very end/beginning (complement?)
Liftoff did not detect YRF1 on chromosome V. 

Is it a valid ORF, is there easy tool to check that?

# Chromosome I

In [65]:
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YAL063C_FLO9_genomic.fsa', out='Blast_results/YAL063C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YAR027W_UIP3_genomic.fsa', out='Blast_results/YAR027W.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)

STDOUT: 
STDERR: 
STDOUT: 
STDERR: 


FLO9:
2 matches: score    e-val
tig00000033 I 2691    0.0  12 gap   86% identity
tig00000017 VIII  1718    0.0  57 gap 

UIP3: 
2 matches
tig00000033 I  233     3e-61  87% identity 6 gap alignment without stop codon at the end.
tig00004928 VII  171     3e-42  82% 6 gap, alignment finish with stop codon

# Chromosome II

In [66]:
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YBL108C-A_PAU9_genomic.fsa', out='Blast_results/YBL108C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YBL111C_YBL111C_genomic.fsa', out='Blast_results/YBL111C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YBR298C_MAL31_genomic.fsa', out='Blast_results/YBR298C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)

STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 


In [67]:
verified[verified['Name'] == 'YBL111C']

,Name,Chromosome,Contig,start,end,Alias,Note,Ontology_term,coverage,curie,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs
10046,YBL111C,5,tig00000015,3147,6415,None,Helicase-like protein encoded within the telom...,"GO:0003674,GO:0005739,GO:0008150,SO:0000704",0.977,SGD:S000002151,Helicase-like protein encoded within the telom...,0,None,None,Verified,None,0.957,0


# Duplicated genes???



In [75]:
# Many duplicated genes in this version
gene_attr['extra_copy_number'].value_counts()

0    6563
1      94
2      22
3      12
4       7
5       5
6       2
Name: extra_copy_number, dtype: int64

In [77]:
gene_attr[gene_attr['extra_copy_number'] > '1']

,seq_id,source,type,start,end,score,strand,phase,attributes,Alias,...,curie,dbxref,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs
9,chrII-2_tig00004927,Liftoff,gene,14242,14700,.,-,.,ID=YGR293C_2;Name=YGR293C;Ontology_term=GO:000...,None,...,SGD:S000003525,SGD:S000003525,Dubious%20open%20reading%20frame,2,None,None,Dubious,None,0.989,0
21,chrII-2_tig00004927,Liftoff,gene,17994,18290,.,-,.,ID=YER188C-A_3;Name=YER188C-A;Ontology_term=GO...,None,...,SGD:S000028764,SGD:S000028764,Putative%20protein%20of%20unknown%20function,3,None,None,Uncharacterized,None,0.943,1
24,chrIII-2_tig00004935,Liftoff,gene,767,1021,.,+,.,ID=YNR077C_3;Name=YNR077C;Ontology_term=GO:000...,None,...,SGD:S000005360,SGD:S000005360,Protein%20of%20unknown%20function%2C%20abundan...,3,None,None,Uncharacterized,None,0.918,1
27,chrIII-2_tig00004935,Liftoff,gene,1323,2063,.,+,.,ID=YCL076W_2;Name=YCL076W;Ontology_term=GO:000...,None,...,SGD:S000000581,SGD:S000000581,Dubious%20open%20reading%20frame,2,None,None,Dubious,None,0.981,0
30,chrIII-2_tig00004935,Liftoff,gene,6426,8271,.,-,.,ID=YCL073C_2;Name=YCL073C;gene=GEX1;Alias=GEX1...,"GEX1,glutathione%20exchanger",...,SGD:S000000575,SGD:S000000575,Proton:glutathione%20antiporter,2,GEX1,None,Verified,None,0.990,0
33,chrIII_tig00004936,Liftoff,gene,760,1014,.,+,.,ID=YNR077C_2;Name=YNR077C;Ontology_term=GO:000...,None,...,SGD:S000005360,SGD:S000005360,Protein%20of%20unknown%20function%2C%20abundan...,2,None,None,Uncharacterized,None,0.918,1
36,chrIII_tig00004936,Liftoff,gene,1316,2055,.,+,.,ID=YCL076W_3;Name=YCL076W;Ontology_term=GO:000...,None,...,SGD:S000000581,SGD:S000000581,Dubious%20open%20reading%20frame,3,None,None,Dubious,None,0.979,0
651,chrIII_tig00004936,Liftoff,gene,310848,311102,.,-,.,ID=YNL337W_4;Name=YNL337W;Ontology_term=GO:000...,None,...,SGD:S000005281,SGD:S000005281,Dubious%20open%20reading%20frame,4,None,None,Dubious,None,0.949,1
2097,chrII_tig00000010,Liftoff,gene,789332,789467,.,+,.,ID=YHL048C-A_6;Name=YHL048C-A;Ontology_term=GO...,None,...,SGD:S000028829,SGD:S000028829,Putative%20protein%20of%20unknown%20function,6,None,None,Uncharacterized,None,0.890,0
2106,chrIV_tig00000001,Liftoff,gene,1219,1356,.,-,.,ID=YHL048C-A_4;Name=YHL048C-A;Ontology_term=GO...,None,...,SGD:S000028829,SGD:S000028829,Putative%20protein%20of%20unknown%20function,4,None,None,Uncharacterized,None,0.920,0


In [70]:
gene_attr[gene_attr.duplicated(subset=['Name'], keep=False) == True]

,seq_id,source,type,start,end,score,strand,phase,attributes,Alias,...,curie,dbxref,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs
0,chrII-2_tig00004927,Liftoff,gene,8297,10119,.,-,.,ID=YBR298C_1;Name=YBR298C;gene=MAL31;Alias=MAL...,"MAL31,MAL3T,MALT,maltose%20permease",...,SGD:S000000502,SGD:S000000502,Maltose%20permease,1,MAL31,None,Verified,None,0.946,0
3,chrII-2_tig00004927,Liftoff,gene,10675,10894,.,-,.,ID=YBR298C-A_1;Name=YBR298C-A;Ontology_term=GO...,None,...,SGD:S000028606,SGD:S000028606,Putative%20protein%20of%20unknown%20function,1,None,None,Uncharacterized,None,0.981,0
6,chrII-2_tig00004927,Liftoff,gene,10985,12717,.,+,.,ID=YBR299W_1;Name=YBR299W;gene=MAL32;Alias=MAL...,"MAL32,MAL3S,MALS,alpha-glucosidase%20MAL32",...,SGD:S000000503,SGD:S000000503,Maltase%20%28alpha-D-glucosidase%29,1,MAL32,None,Verified,None,0.981,0
9,chrII-2_tig00004927,Liftoff,gene,14242,14700,.,-,.,ID=YGR293C_2;Name=YGR293C;Ontology_term=GO:000...,None,...,SGD:S000003525,SGD:S000003525,Dubious%20open%20reading%20frame,2,None,None,Dubious,None,0.989,0
12,chrII-2_tig00004927,Liftoff,gene,14660,15022,.,+,.,ID=YGR294W_1;Name=YGR294W;gene=PAU12;Alias=PAU...,"PAU12,seripauperin%20PAU12",...,SGD:S000003526,SGD:S000003526,Protein%20of%20unknown%20function,1,PAU12,None,Verified,None,0.983,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20903,chrX_tig00001098,Liftoff,gene,17569,18158,.,+,.,ID=YJL218W_1;Name=YJL218W;Alias=acetyltransfer...,acetyltransferase,...,SGD:S000003754,SGD:S000003754,Mitochondrial%20protein%2C%20putative%20acetyl...,1,None,None,Uncharacterized,None,0.996,0
20906,chrX_tig00001098,Liftoff,gene,18728,19322,.,+,.,ID=YJL217W_1;Name=YJL217W;gene=REE1;Alias=REE1...,REE1,...,SGD:S000003753,SGD:S000003753,Cytoplasmic%20protein%20involved%20in%20the%20...,1,REE1,None,Verified,None,0.993,0
22173,chrX_tig00001098,Liftoff,gene,712194,713216,.,-,.,"ID=YFL058W;Name=YFL058W;gene=THI5;Alias=THI5,4...","THI5,4-amino-5-hydroxymethyl-2-methylpyrimidin...",...,SGD:S000001836,SGD:S000001836,Protein%20involved%20in%20synthesis%20of%20the...,0,THI5,None,Verified,None,0.990,1
22212,chrX_tig00001098,Liftoff,gene,720334,720468,.,+,.,ID=YHL048C-A;Name=YHL048C-A;Ontology_term=GO:0...,None,...,SGD:S000028829,SGD:S000028829,Putative%20protein%20of%20unknown%20function,0,None,None,Uncharacterized,None,0.955,1


Duplicated genes:

ATP8/Q0080: Subunit 8 of the F0 sector of mitochondrial F1F0 ATP synthase; encoded on the mitochondrial genome; ATP8 and ATP6 mRNAs are not translated in the absence of the F1 sector of ATPase

OLI1/Q0130: F0-ATP synthase subunit c (ATPase-associated proteolipid); encoded on the mitochondrial genome; mutation confers oligomycin resistance; expression is specifically dependent on the nuclear genes AEP1 and AEP2 1 2 3 4
Name Description
OLIgomycin resistance 1

Both on mitochondria so not that relevant to my project.

PAU9/YBL108C-A: Protein of unknown function; member of the seripauperin multigene family encoded mainly in subtelomeric regions; SWAT-GFP and mCherry fusion proteins localize to the endoplasmic reticulum and vacuole respectively

YHR213W-A: Putative protein of unknown function; identified by gene-trapping, microarray-based expression analysis, and genome-wide homology searching 